In [1]:
!pip install git+https://github.com/PyFstat/PyFstat@python37

  Cloning https://github.com/PyFstat/PyFstat (to revision python37) to /tmp/pip-req-build-y34t532e
  Running command git clone --filter=blob:none --quiet https://github.com/PyFstat/PyFstat /tmp/pip-req-build-y34t532e
  Running command git checkout -b python37 --track origin/python37
  Switched to a new branch 'python37'
  Branch 'python37' set up to track remote branch 'python37' from 'origin'.
  Resolved https://github.com/PyFstat/PyFstat to commit 73ad1acdc9385a234727abf3eb9f93c9298fc5e5
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 396.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.0 MB/s eta 0:00:00
  Installing build dependencies ... - \ |

In [2]:
!pip install h5py

In [3]:
import uuid
import h5py
import numpy as np
import pandas as pd
import torch as tc
import random
import tqdm
import pyfstat,sys,os,io,cv2,warnings
from pyfstat.utils import get_sft_as_arrays
from typing import TYPE_CHECKING, Iterable, Optional
import logging,shutil,tqdm
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [4]:
def data_gen(signal_rate):
    sft_path = []
    pi = 3.141592

    tstart = 1238166018
    totaldiff = 200000
    tdiff = int(random.random() * totaldiff) - (totaldiff / 2)
    tstart = tstart - tdiff

    f0 = random.random() * (500 - 50) + 50
    h0 = random.random() * (100 - 10) + 10

    f1 = random.random() * (1.0e-8 - 1.0e-12) + 1.0e-12
    alpha = random.random() * (2*pi - 0) + 0
    delta = random.random() * ((pi/2) - (-pi/2)) + (-pi/2)
    cosi = random.random() * (1 - (-1)) + (-1)
    psi = random.random() * ((pi/4) - (-pi/4)) + (-pi/4)
    phi = random.random() * ((2*pi) - (0)) + (0)

    sqrtSX = 5e-24

    band = 0.36
    tdur = 7.5 * 86400
    signal_kwargs = {
        "outdir": "gen_outdir",
        "tstart": tstart,
        "duration": tdur,
        "detectors": "H1,L1",
        "Tsft": 1800,
        "Band": band,

        "sqrtSX": sqrtSX,
        "SFTWindowType": "tukey", 
        "SFTWindowBeta": 0.01,

        "F0": f0, # 50 to 500
    }

    signal_writer = pyfstat.Writer( **signal_kwargs )
    signal_writer.make_data()        
    sft_path.append(signal_writer.sftfilepath)

    freqs, times, sft_data = get_sft_as_arrays(";".join(sft_path))
    if signal_rate == 0:
        return freqs, times, sft_data
    else:
        signal_kwargs2 = {
            "outdir": "gen_outdir",
            "tstart": tstart,
            "duration": 7.5 * 86400,
            "detectors": "H1,L1",
            "Tsft": 1800,
            "Band": band, 

            "sqrtSX": 0,
            "SFTWindowType": "tukey", 
            "SFTWindowBeta": 0.01,

            "F0": f0, # 50 to 500

            "h0": sqrtSX / h0, # div by 10 to 100
            "F1": f1, # 1.0e-12 to 1.0e-8 (or so; could be to 0)
            "Alpha": alpha, # 0 to 2pi
            "Delta": delta, # -pi/2 to pi/2
            "cosi": cosi, # -1.0 to 1.0
            "psi": psi, # any (mapped -pi/4 to pi/4)
            "phi": phi, # any (mapped 0 to 2pi)
            
            "noiseSFTs": ";".join(sft_path)

        }
        signal_writer = pyfstat.Writer( **signal_kwargs2 )
        signal_writer.make_data()
        signal_sft_path = signal_writer.sftfilepath
        
        freqs, times, sft_data = get_sft_as_arrays(signal_sft_path)
        return freqs, times, sft_data

In [5]:
file_path="/kaggle/working/"
try:
    os.mkdir(path=file_path+"data_gen")
except:
    shutil.rmtree(file_path+"data_gen") #delete folder
    os.mkdir(path=file_path+"data_gen")

In [6]:
data_gen_labels = pd.DataFrame()

In [7]:
idd = []
target = []
total = 2000
time_t = tqdm.tqdm(total=total)
for _ in range(total):
    ids = uuid.uuid4().hex[:9]     
    with h5py.File("data_gen/"+str(ids)+".hdf5", 'w') as hf:
        if random.random() < 0.5:
            freq, time, fourier_data = data_gen(1)
            tar = 1
        else:
            freq, time, fourier_data = data_gen(0)
            tar = 0
        g = hf.create_group(str(ids))
        g.create_dataset('frequency_Hz', data=freq)
        g1 = g.create_group('H1')
        g2 = g.create_group('L1')
        g1.create_dataset('SFTs', data=fourier_data['H1'])
        g1.create_dataset('timestamps_GPS', data=time['H1'])
        g2.create_dataset('SFTs', data=fourier_data['L1'])
        g2.create_dataset('timestamps_GPS', data=time['L1'])
    idd.append(ids)
    target.append(tar)
    time_t.update(1)

100%|██████████| 2000/2000 [1:02:54<00:00,  1.56s/it]

In [8]:
data_gen_labels['id'] = idd
data_gen_labels['target'] = target

In [9]:
data_gen_labels.to_csv('data_gen_labels.csv', index=False)

In [10]:
shutil.rmtree(file_path+'gen_outdir')